## качаем данные

In [1]:
import requests
from bs4 import BeautifulSoup 
import time
from urllib.parse import urlparse, urljoin
from tqdm.notebook import tqdm

In [ ]:
## найти id каждого вопроса на страницах
page = 1
main_url = 'https://zpp.rospotrebnadzor.ru/Forum/Appeals/AjaxindexList?page={page}&searchtext=&categories=[]'

def ids_from_page(soup):
    return [i.get('href') for i  in soup.findAll('a', {'class':'appeal-title-link'})]
    
urls = []
for page in tqdm(range(1, 500), desc='Pages'):
    time.sleep(0.5)
    res = requests.get(main_url.format(page=page))  
    soup = BeautifulSoup(res.text)
    urls.extend(ids_from_page(soup))

Pages:   0%|          | 0/499 [00:00<?, ?it/s]

# Новый раздел

In [ ]:
## имея id запросов достанем тексты на страницах
domain = 'https://zpp.rospotrebnadzor.ru'

def get_text_page_question(soup):
    return soup.findAll('p', {'class':'appeal-details-message'})[0].text
def get_text_page_answer(soup):
    return soup.findAll('p', {'class':'appeal-comments-message'})[0].text

question = []
answers = []

for url in tqdm(urls, desc='Pages'):
    time.sleep(0.5)
    res = requests.get(urljoin(domain, url))  
    soup = BeautifulSoup(res.text)
    
    question.append(get_text_page_question(soup))
    try:
        answers.append(get_text_page_answer(soup))
    except:
        answers.append('No answer')

Pages:   0%|          | 0/2495 [00:00<?, ?it/s]

In [ ]:
## посмотрим, что скачалось
N = 2
print('Q:', question[N])
print()
print('A:', answers[N])

Q: Добрый день! Меня зовут Евгения. 25.12.2021г. на сервисе Рамблер Касса приобрела билеты на концерт Imagine Dragons, который должен был состояться в Санкт-Петербурге 07.06.2022г.  15 марта 2022г. пришло сообщение от вышеупомянутой организации о том, что концерт отменили. Я написала письмо с просьбой вернуть деньги за билеты, сегодня 16.06.2022г.  - деньги ещё не вернули, сроки возврата не называют. Я им писала повторно, но в ответ только "Ваша заявка в работе". Все подтверждающие документы могу предоставить (чек, все переписки, уведомление об отмене концерта). Прошу помочь разобраться в данной ситуации. 

A: Ваш вопрос носит индивидуальный характер, не забудьте приложить договор, чек, претензию ( в которой указаны Ваши требования и ответ на неё при наличии).
Раздел «Виртуальная приёмная» ГИР ЗПП представляет собой площадку для общения между ПОТРЕБИТЕЛЕМ и Роспотребнадзором. Ваше обращение может быть рассмотрено в рамках полномочий Роспотребнадзора в соответствии с требованиями Федера

In [2]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame({'q':question})
print (df)

                                                      q
0     Здравствуйте. Купил телефон на алиэкспресс. На...
1     Добрый день.  Планирую открыть магазин или лав...
2     Добрый день! Меня зовут Евгения. 25.12.2021г. ...
3     Добрый день! В ходе поиска проведения отпуска ...
4     На сайте wildberries.ru продавец Данилов Дмитр...
...                                                 ...
2490  В г.Чита Забайкальского края в магазине Светоф...
2491  Добрый день, купила телефон через интернет-маг...
2492  20.10.21 г. Сделан заказ в интернет магазине u...
2493  Прошу разъяснить п2516 действующих СП, который...
2494  Добрый день!\r\nПодскажите, пожалуйста, какую ...

[2495 rows x 1 columns]


In [3]:
import pickle

In [ ]:
file_result= open('qa500.dat','ab')
pickle.dump(df,file_result)
file_result.close()

In [ ]:
search = '@[\w]*'
df['q'].str.findall(search)

0             []
1        [@mail]
2             []
3             []
4             []
          ...   
2490          []
2491          []
2492    [@gmail]
2493          []
2494          []
Name: q, Length: 2495, dtype: object

In [ ]:
df['q'] = df['q'].str.replace(search, ' ')

In [ ]:
df['q'] = df['q'].str.lower()

In [ ]:
def jf(x):
    return " ".join([w for w in x.split() if len(w)>1])

In [ ]:
df['q'] = df['q'].apply(jf)

In [ ]:
search = r'[^а-яА-ЯёЁa-zA-Z]'
df['q'] = df['q'].str.replace(search, ' ')

In [ ]:
df['q'][:5]

0    здравствуйте  купил телефон на алиэкспресс  на...
1    добрый день  планирую открыть магазин или лавк...
2    добрый день  меня зовут евгения            г  ...
3    добрый день  ходе поиска проведения отпуска на...
4    на сайте wildberries ru продавец данилов дмитр...
Name: q, dtype: object

In [4]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool

In [5]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.4 MB/s 
     |████████████████████████████████| 8.2 MB 8.7 MB/s 


In [6]:
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

# mystem = Mystem()
# def lemmatize(words, lemmer = mystem, stopwords = stopwords_list):
#     lemmas = lemmer.lemmatize(' '.join(words))
#     return [w for w in lemmas if not w in stopwords 
#             and w.isalpha()]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [ ]:
df['q'].iloc[1]

'добрый день  планирую открыть магазин или лавку по продаже пищевого льда  также планирую его производство маленьких масштабах продажу по магазинам  как получить сертификат качества льда  для продаж  магазины без сертификата не принимают  как получить сертификат качества  какая аренда помещения должна  как получить маркировку упаковки  чего все начать  живу сочи планирую тут открыться  эл почта kurort    ru спасибо за ответ '

In [ ]:
print(preprocess(df['q'].iloc[1]))

['добрый', 'день', 'планировать', 'открыть', 'магазин', 'лавка', 'продажа', 'пищевой', 'лёд', 'также', 'планировать', 'производство', 'маленький', 'масштаб', 'продажа', 'магазин', 'получить', 'сертификат', 'качество', 'лёд', 'продажа', 'магазин', 'сертификат', 'принимать', 'получить', 'сертификат', 'качество', 'аренда', 'помещение', 'должный', 'получить', 'маркировка', 'упаковка', 'всё', 'начать', 'жить', 'сочи', 'планировать', 'открыться', 'почта', 'kurort', 'спасибо', 'ответ']


In [ ]:
preprocessed_text = list(tqdm(map(preprocess, df['q']), total=len(df['q'])))

  0%|          | 0/2495 [00:00<?, ?it/s]

In [ ]:
df['q_text'] = preprocessed_text
df['q_text'].sample(3)

2263    [купить, интернет, магазин, набор, долговремен...
1276    [здравствуйте, гражданка, финляндия, прописать...
1091    [добрый, день, оплатить, товар, сумма, рубль, ...
Name: q_text, dtype: object

In [ ]:
file_result= open('qa500_text.dat','ab')
pickle.dump(df,file_result)
file_result.close()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/qa500_text.dat')

In [9]:
df['q_text'].sample(3)

2286    [митягин, артём, олегович, ооо, мвм, представи...
901     [добрый, день, звать, нина, тур, агентство, ан...
258     [здравствуйте, подвал, жилой, дом, энтузиаст, ...
Name: q_text, dtype: object

In [ ]:
!apt-get --yes update
!apt-get --yes install git make cmake build-essential libboost-all-dev
!apt-get --yes install python-numpy python-pandas python-scipy
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!pip install protobuf tqdm wheel
!git clone --branch=stable https://github.com/bigartm/bigartm.git

In [11]:
%cd bigartm

/content/bigartm


In [12]:
!mkdir build

In [13]:
%cd build

/content/bigartm/build


In [ ]:
!cmake ..

In [ ]:
!make

In [ ]:
!make install

In [17]:
!pip install python/bigartm*.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./python/bigartm-0.10.0-cp37-cp37m-linux_x86_64.whl


In [18]:
import artm

In [19]:
artm.version()

'0.10.0'

In [20]:
artm.ARTM(num_topics=10).info

config {
  topic_name: "topic_0"
  topic_name: "topic_1"
  topic_name: "topic_2"
  topic_name: "topic_3"
  topic_name: "topic_4"
  topic_name: "topic_5"
  topic_name: "topic_6"
  topic_name: "topic_7"
  topic_name: "topic_8"
  topic_name: "topic_9"
  score_config {
    name: "^^^ItemsProcessedScore^^^"
    type: ScoreType_ItemsProcessed
    config: ""
  }
  pwt_name: "pwt"
  nwt_name: "nwt"
  num_document_passes: 10
  reuse_theta: false
  cache_theta: false
}
score {
  name: "^^^ItemsProcessedScore^^^"
  type: "N4artm5score14ItemsProcessedE"
}
processor_queue_size: 0
num_processors: 2

In [21]:
df['q_text'] = df['q_text'].apply(lambda x: ' '.join(x))

In [22]:
i = 0
d = df.iloc[i]
d

q         здравствуйте  купил телефон на алиэкспресс  на...
q_text    здравствуйте купить телефон алиэкспресс посылк...
Name: 0, dtype: object

In [23]:
f'doc{i} {d.q_text}'

'doc0 здравствуйте купить телефон алиэкспресс посылка написать полный стоимость телефон трек номер отслеживание посылка прийти отделение связь номер засветиться готовый выдача прийти отделение посылка отдать формулировка наслать горячий линия почта россия говорить посылка отделение факт отрицать всё дешёвый посылка получить дорогой пропасть причём отделение розыск подать'

In [24]:
def convert_to_vw_format(index, row):
    return f'doc{index} {row.text}'

In [25]:
def to_vw_format(document):
    return '|text ' + ' '.join(word for word in document.split())

f = open('documents.txt', 'w')
vw = df['q_text'].apply(to_vw_format)
for i in range(0,len(vw)):
  f.write(str(vw.values[i])+ '\n')
f.close()

In [26]:
batch_vectorizer = artm.BatchVectorizer(data_path='documents.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder='batch_vect')

In [27]:
%%time 
lda = artm.LDA(num_topics=10, alpha=0.1, beta=0.001, 
               cache_theta=True,
               num_document_passes=5, 
               dictionary=batch_vectorizer.dictionary)

lda.fit_offline(batch_vectorizer=batch_vectorizer, 
                num_collection_passes=10)

CPU times: user 4.24 s, sys: 57.6 ms, total: 4.29 s
Wall time: 2.96 s


In [28]:
lda.perplexity_value

[13665.2265625,
 2159.31005859375,
 1993.734130859375,
 1793.1693115234375,
 1641.355224609375,
 1537.19775390625,
 1466.5108642578125,
 1420.4947509765625,
 1389.599365234375,
 1367.3133544921875]

In [29]:
np.log(lda.perplexity_last_value)

7.220603038320792

In [30]:
top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
     print ('Topic #{0}: {1}'.format(i, token_list))

Topic #0: ['это', 'магазин', 'продукция', 'торговый', 'просить', 'который', 'водитель', 'ребёнок', 'маска', 'продажа']
Topic #1: ['дом', 'это', 'год', 'услуга', 'договор', 'просить', 'который', 'день', 'компания', 'адрес']
Topic #2: ['руб', 'день', 'сказать', 'добрый', 'здравствуйте', 'дом', 'дать', 'делать', 'который', 'мочь']
Topic #3: ['номер', 'средство', 'банк', 'деньга', 'день', 'карта', 'который', 'это', 'дать', 'денежный']
Topic #4: ['магазин', 'товар', 'возврат', 'день', 'продавец', 'вернуть', 'купить', 'средство', 'денежный', 'качество']
Topic #5: ['услуга', 'это', 'потребитель', 'право', 'договор', 'закон', 'ооо', 'ребёнок', 'защита', 'тур']
Topic #6: ['день', 'магазин', 'это', 'просить', 'средство', 'возврат', 'товар', 'договор', 'рубль', 'который']
Topic #7: ['товар', 'заказ', 'магазин', 'доставка', 'день', 'интернет', 'оплатить', 'деньга', 'сайт', 'возврат']
Topic #8: ['просить', 'санитарный', 'здравствуйте', 'площадка', 'россия', 'день', 'мусор', 'область', 'территория',

In [31]:
plsa = artm.ARTM(num_topics=10, 
                 dictionary=batch_vectorizer.dictionary)

In [32]:
plsa.scores.add(artm.PerplexityScore(name='perplexity_score',
                 dictionary=batch_vectorizer.dictionary))

In [33]:
plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', num_tokens=6))

In [34]:
plsa.fit_offline(batch_vectorizer=batch_vectorizer,
                 num_collection_passes=20)

In [35]:
plsa.score_tracker['perplexity_score'].value

[13419.6064453125,
 1997.908935546875,
 1721.782470703125,
 1522.435791015625,
 1410.9764404296875,
 1346.20556640625,
 1304.9324951171875,
 1276.9993896484375,
 1257.539794921875,
 1243.5697021484375,
 1232.90185546875,
 1224.2962646484375,
 1216.9378662109375,
 1210.664794921875,
 1205.649169921875,
 1201.3760986328125,
 1197.399169921875,
 1194.310791015625,
 1191.765869140625,
 1189.37744140625]

In [36]:
for topic_name in plsa.topic_names:
    print (topic_name + ': ', plsa.score_tracker['TopTokensScore'].last_tokens[topic_name])

topic_0:  ['магазин', 'это', 'просить', 'маска', 'продукция', 'который']
topic_1:  ['дом', 'квартира', 'адрес', 'вода', 'просить', 'год']
topic_2:  ['день', 'тест', 'ребёнок', 'результат', 'пцр', 'это']
topic_3:  ['средство', 'возврат', 'денежный', 'билет', 'день', 'ооо']
topic_4:  ['товар', 'магазин', 'продавец', 'возврат', 'день', 'вернуть']
topic_5:  ['договор', 'услуга', 'право', 'потребитель', 'это', 'компания']
topic_6:  ['карта', 'номер', 'банк', 'деньга', 'это', 'рубль']
topic_7:  ['заказ', 'товар', 'доставка', 'день', 'оплатить', 'магазин']
topic_8:  ['просить', 'ребёнок', 'санитарный', 'здравствуйте', 'код', 'россия']
topic_9:  ['сказать', 'ремонт', 'это', 'телефон', 'всё', 'центр']


In [37]:
batch_vectorizer.dictionary.filter(min_tf=10, max_tf=5000)

artm.Dictionary(name=9007b097-78a6-4cd6-a592-652fc167c8d5, num_entries=2294)

In [38]:
plsa = artm.ARTM(num_topics=10, 
                 dictionary=batch_vectorizer.dictionary)
plsa.scores.add(artm.PerplexityScore(name='perplexity_score',
                dictionary=batch_vectorizer.dictionary))

plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', 
                                    num_tokens=6))
plsa.fit_offline(batch_vectorizer=batch_vectorizer, 
                 num_collection_passes=20)

In [39]:
plsa.score_tracker['perplexity_score'].value

[620.3570556640625,
 306.8083190917969,
 285.1766662597656,
 262.5078125,
 245.87979125976562,
 234.81906127929688,
 227.68077087402344,
 222.93760681152344,
 219.6274871826172,
 217.23915100097656,
 215.48191833496094,
 214.131591796875,
 213.07382202148438,
 212.23731994628906,
 211.5559539794922,
 211.00204467773438,
 210.56539916992188,
 210.2004852294922,
 209.8823699951172,
 209.60118103027344]

In [40]:
for topic_name in plsa.topic_names:
    print (topic_name + ': ', plsa.score_tracker['TopTokensScore'].last_tokens[topic_name])

topic_0:  ['право', 'документ', 'закон', 'продажа', 'договор', 'потребитель']
topic_1:  ['дом', 'адрес', 'просить', 'квартира', 'вода', 'наш']
topic_2:  ['это', 'сказать', 'день', 'всё', 'тест', 'результат']
topic_3:  ['средство', 'возврат', 'денежный', 'день', 'билет', 'ооо']
topic_4:  ['магазин', 'товар', 'продавец', 'возврат', 'день', 'цена']
topic_5:  ['услуга', 'договор', 'компания', 'оплата', 'это', 'рубль']
topic_6:  ['карта', 'номер', 'банк', 'деньга', 'это', 'счёт']
topic_7:  ['заказ', 'товар', 'доставка', 'сайт', 'день', 'интернет']
topic_8:  ['ребёнок', 'день', 'детский', 'код', 'просить', 'добрый']
topic_9:  ['телефон', 'день', 'ремонт', 'деньга', 'сказать', 'центр']
